# 1. Environment Setting

In [1]:
import torch
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 2.0.1+cu118


In [2]:
!pip install -q dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 33.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 43.9 MB/s eta 0:

In [3]:
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import time
from dgl.nn import GraphConv, SAGEConv, GATConv, HeteroGraphConv

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

# 2. Build Graph

In [5]:
# Assign node, edge
n_users = 1000 #node
n_items = 500 #node
n_follows = 3000 #edge
n_clicks = 5000 #edge
n_dislikes = 500 #edge

In [6]:
# Assign Parameter
n_hetero_features = 10
n_user_classes = 5
n_max_clicks = 10

In [7]:
# Create random edges
follow_src = np.random.randint(0, n_users, n_follows)
follow_dst = np.random.randint(0, n_users, n_follows)
click_src = np.random.randint(0, n_users, n_clicks)
click_dst = np.random.randint(0, n_items, n_clicks)
dislike_src = np.random.randint(0, n_users, n_dislikes)
dislike_dst = np.random.randint(0, n_items, n_dislikes)

In [8]:
#np.random.randint() 0부터 n_users까지 정수 중 n_follows개의 랜덤 정수
follow_src=np.random.randint(0,n_users,n_follows)
follow_src

array([770,  98, 217, ..., 358, 314, 911])

In [9]:
hetero_graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'click', 'item'): (click_src, click_dst),
    ('item', 'clicked-by', 'user'): (click_dst, click_src),
    ('user', 'dislike', 'item'): (dislike_src, dislike_dst),
    ('item', 'disliked-by', 'user'): (dislike_dst, dislike_src)})

### 선언한 그래프가 GPU에서 동작하도록 선언

In [10]:
hetero_graph = hetero_graph.to(device)

In [11]:
hetero_graph

Graph(num_nodes={'item': 500, 'user': 1000},
      num_edges={('item', 'clicked-by', 'user'): 5000, ('item', 'disliked-by', 'user'): 500, ('user', 'click', 'item'): 5000, ('user', 'dislike', 'item'): 500, ('user', 'follow', 'user'): 3000, ('user', 'followed-by', 'user'): 3000},
      metagraph=[('item', 'user', 'clicked-by'), ('item', 'user', 'disliked-by'), ('user', 'item', 'click'), ('user', 'item', 'dislike'), ('user', 'user', 'follow'), ('user', 'user', 'followed-by')])

# 3. Assign Features

In [12]:
# Assign Parameter
n_hetero_features = 256
n_user_classes = 5
n_max_clicks = 10

In [ ]:
#torch.randn(): normal distribution을 따르는 난수 생성(size 크기)

In [13]:
# Assign Node Features
#Cannot assign node feature "feature" on device cpu to a graph on device cuda:0. Call DGLGraph.to() to copy the graph to the same device.

hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users, n_hetero_features).to(device)
hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items, n_hetero_features).to(device)

In [14]:
# Assign Label Data
hetero_graph.nodes['user'].data['label'] = torch.randint(0, n_user_classes, (n_users,)).to(device)

In [ ]:
# Randomly generate training masks on user nodes and click edges

# hetero_graph.nodes['user'].data['train_mask'] = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6).to(device)

In [17]:
train_list = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6).to(device).tolist()

In [19]:
test_list = []
for i in train_list:
  if i==False:
    test_list.append(True)
  else:
    test_list.append(False)
len(test_list)

1000

In [22]:
hetero_graph.nodes['user'].data['train_mask'] =  torch.BoolTensor(train_list).to(device)
hetero_graph.nodes['user'].data['test_mask'] =  torch.BoolTensor(test_list).to(device)

# 4. Define Model

In [23]:
# Define a Heterograph Conv model
class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.conv1 = HeteroGraphConv({
            rel: GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        
        self.conv2 = HeteroGraphConv({
            rel: GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [24]:
model = RGCN(n_hetero_features, 128, n_user_classes, hetero_graph.etypes).to(device) #model = Model().to(device)

In [33]:
# 학습에 필요한 데이터 할당 (목적: 유저 클래스 분류)
user_feats = hetero_graph.nodes['user'].data['feature']
item_feats = hetero_graph.nodes['item'].data['feature']
labels = hetero_graph.nodes['user'].data['label']
train_mask = hetero_graph.nodes['user'].data['train_mask']
test_mask = hetero_graph.nodes['user'].data['test_mask']

In [26]:
hetero_graph.device, item_feats.device, user_feats.device

(device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0))

In [27]:
node_features = {'user': user_feats, 'item': item_feats}
h_dict = model(hetero_graph, {'user': user_feats, 'item': item_feats})
h_user = h_dict['user']
h_item = h_dict['item']

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \


In [30]:
logits = model(hetero_graph, node_features)['user']

In [31]:
logits[train_mask]

tensor([[-0.4516,  2.1322,  0.6942,  1.6501, -1.4908],
        [ 1.7404,  0.0233, -0.5368, -0.2783, -0.2243],
        [ 0.1508, -0.1655, -0.1816,  0.4062, -0.7211],
        ...,
        [ 0.5730,  0.4671,  0.5902,  0.8492, -0.5441],
        [ 0.3838, -0.1043, -2.4252,  1.5405, -0.8883],
        [ 0.2225,  1.1439,  0.6740,  0.5679, -1.0437]], device='cuda:0',
       grad_fn=<IndexBackward0>)

In [40]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

for epoch in range(500):
    model.train()
    # forward propagation by using all nodes and extracting the user embeddings
    logits = model(hetero_graph, node_features)['user']
    
    # compute loss
    train_loss = criterion(logits[train_mask], labels[train_mask])
    if epoch % 10 == 0:
        print(train_loss.item())
        
    # backward propagation
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # evaluation
    num_correct = 0
    num_tests = 0
    if epoch % 10 == 0:
      model.eval()
      with torch.no_grad():
        logits = model(hetero_graph, node_features)['user']
        val_loss = criterion(logits[test_mask], labels[test_mask])
        acc = (logits[test_mask].argmax(dim=1) == labels[test_mask]).float().mean().item()
        print( "Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} ".format(
        epoch, val_loss.item(), acc))

0.0
Epoch 00000 | Loss 8.6878 | Accuracy 0.2136 
0.0
Epoch 00010 | Loss 8.6908 | Accuracy 0.2161 
0.0
Epoch 00020 | Loss 8.6939 | Accuracy 0.2161 
0.0
Epoch 00030 | Loss 8.6969 | Accuracy 0.2161 
0.0
Epoch 00040 | Loss 8.6999 | Accuracy 0.2161 
0.0
Epoch 00050 | Loss 8.7029 | Accuracy 0.2161 
0.0
Epoch 00060 | Loss 8.7059 | Accuracy 0.2161 
0.0
Epoch 00070 | Loss 8.7089 | Accuracy 0.2161 
0.0
Epoch 00080 | Loss 8.7119 | Accuracy 0.2161 
0.0
Epoch 00090 | Loss 8.7149 | Accuracy 0.2161 
0.0
Epoch 00100 | Loss 8.7179 | Accuracy 0.2161 
0.0
Epoch 00110 | Loss 8.7208 | Accuracy 0.2161 
0.0
Epoch 00120 | Loss 8.7238 | Accuracy 0.2161 
0.0
Epoch 00130 | Loss 8.7267 | Accuracy 0.2161 
0.0
Epoch 00140 | Loss 8.7297 | Accuracy 0.2161 
0.0
Epoch 00150 | Loss 8.7326 | Accuracy 0.2161 
0.0
Epoch 00160 | Loss 8.7356 | Accuracy 0.2161 
0.0
Epoch 00170 | Loss 8.7385 | Accuracy 0.2161 
0.0
Epoch 00180 | Loss 8.7414 | Accuracy 0.2161 
0.0
Epoch 00190 | Loss 8.7443 | Accuracy 0.2161 
0.0
Epoch 00200 | Lo